In [ ]:
def setup_notebook(fix_python_path=True, reduce_margins=True, plot_inline=True):
    if reduce_margins:
        # Reduce side margins of the notebook
        from IPython.core.display import display, HTML
        display(HTML("<style>.container { width:100% !important; }</style>"))

    if fix_python_path:
        # add egosocial to the python path
        import os, sys
        sys.path.extend([os.path.dirname(os.path.abspath('.'))])

    if plot_inline:
        # Plots inside cells
        %matplotlib inline
    
    global __file__
    __file__ = 'Notebook'

setup_notebook()

In [ ]:
import json
import itertools
import os
import pandas as pd
import numpy as np

import egosocial
import egosocial.config

from egosocial.utils.filesystem import list_files_in_segment, list_segments
from egosocial.utils.parser import FACE_DETECTION, load_faces_from_file

In [ ]:
def load_dataset_defition(dataset_path):
    with open(dataset_path, 'r') as json_file:
        dataset_def = json.load(json_file)

    # flatten the segments structure
    samples = pd.DataFrame(list(itertools.chain(*dataset_def)))
        
    return samples

def load_faces(base_dir, face_detection_method=FACE_DETECTION.DOCKER_FACE):
    pattern = FACE_DETECTION.get_file_pattern(face_detection_method)

    face_map = {}
    for split in next(os.walk(base_dir))[1]:
        detection_dir = os.path.join(base_dir, split)
        for segment_id in list_segments(detection_dir):        
            for detection_path in list_files_in_segment(detection_dir,
                                                        segment_id,
                                                        file_pattern=pattern):
                for face in load_faces_from_file(detection_path,
                                                  format=face_detection_method):
                    face_key = (split, segment_id, face.face_id.split('_')[-1])
                    face_map[face_key] = face
    
    return face_map

In [ ]:
BASE_DIR = os.path.join(egosocial.config.TMP_DIR, 'egocentric', 'datasets')
dataset_path = os.path.join(BASE_DIR, 'merged_dataset.json')
detection_dir = os.path.join(BASE_DIR, 'facedocker_detection')
face_height_path = os.path.join(BASE_DIR, 'face_height.csv')

In [ ]:
data = load_dataset_defition(dataset_path)

In [ ]:
face_map = load_faces(detection_dir)

In [ ]:
def extract_face_height(frames, face_map):
    height_values = []
    for row in frames.itertuples():
        face_key = row.split, str(row.segment_id), str(row.face_id)
        face = face_map[face_key]
        height_values.append(face.bbox.height)
    
    return pd.DataFrame(height_values)

In [ ]:
face_height = extract_face_height(data, face_map)

In [ ]:
face_height.to_csv(face_height_path, index=False)

In [ ]:
distance = np.loadtxt(os.path.join(BASE_DIR, 'distance.csv'))
output_path = os.path.join(BASE_DIR, 'distance.npy')
np.save(output_path, distance)